# piplineで一発
Transformers ライブラリで最も基本的なオブジェクトは pipeline() 関数です。これはモデルを必要な前処理と後処理のステップに接続し、任意のテキストを直接入力して理解しやすい答えを得ることを可能にします。

huggingfaceのpiplineはタスクを抽象化したモジュールです。
- タスク名を指定することで、そのタスクを処理できます。タスクに応じたパラメタを受け付けます。
- モデル・トークナイザーも指定することができます。

In [ ]:
!pip install  datasets sentencepiece

# https://stackoverflow.com/questions/76448287/how-can-i-solve-importerror-using-the-trainer-with-pytorch-requires-accele
!pip install -U transformers
!pip install -U accelerate

In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598050713539124}]

In [2]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598050713539124},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [3]:
classifier("AIは人類を滅ぼす")

[{'label': 'NEGATIVE', 'score': 0.9632443785667419}]

このモデルは、分類器オブジェクトを作成する際にダウンロードされ、キャッシュされます。コマンドを再実行すると、キャッシュされたモデルが代わりに使用され、モデルを再度ダウンロードする必要はありません。

パイプラインにテキストを渡す場合、主に3つのステップがあります。

1. テキストは、モデルが理解できる形式に前処理されます。
2. 前処理された入力は、モデルに渡されます。
3. モデルの予測値が後処理され、その結果を理解できるようになります。


現在利用可能なパイプライン（一部）
- feature-extraction (get the vector representation of a text)
- fill-mask
- ner (named entity recognition)
- question-answering
- sentiment-analysis
- summarization
- text-generation
- translation
- zero-shot-classification


In [1]:
from transformers import pipeline
transcriber = pipeline(task="automatic-speech-recognition")

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
transcriber("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/hajime/projects/ai_lab/.venv/lib/python3.9/site-packages/transformers/pipelines/audio_uti │
│ ls.py:32 in ffmpeg_read                                                                          │
│                                                                                                  │
│    29 │   ]                                                                                      │
│    30 │                                                                                          │
│    31 │   try:                                                                                   │
│ ❱  32 │   │   with subprocess.Popen(ffmpeg_command, stdin=subprocess.PIPE, stdout=subprocess.P   │
│    33 │   │   │   output_stream = ffmpeg_process.communicate(bpayload)                           │
│    34 │   except FileNotFoundError as error:                                                     │
│    35 │   │   raise ValueError("ffmpeg was not found but is required to load audio files from    │
│                                                                                                  │
│ /Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib │
│ /python3.9/subprocess.py:951 in __init__                                                         │
│                                                                                                  │
│    948 │   │   │   │   │   self.stderr = io.TextIOWrapper(self.stderr,                           │
│    949 │   │   │   │   │   │   │   encoding=encoding, errors=errors)                             │
│    950 │   │   │                                                                                 │
│ ❱  951 │   │   │   self._execute_child(args, executable, preexec_fn, close_fds,                  │
│    952 │   │   │   │   │   │   │   │   pass_fds, cwd, env,                                       │
│    953 │   │   │   │   │   │   │   │   startupinfo, creationflags, shell,                        │
│    954 │   │   │   │   │   │   │   │   p2cread, p2cwrite,                                        │
│                                                                                                  │
│ /Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib │
│ /python3.9/subprocess.py:1821 in _execute_child                                                  │
│                                                                                                  │
│   1818 │   │   │   │   │   │   err_filename = orig_executable                                    │
│   1819 │   │   │   │   │   if errno_num != 0:                                                    │
│   1820 │   │   │   │   │   │   err_msg = os.strerror(errno_num)                                  │
│ ❱ 1821 │   │   │   │   │   raise child_exception_type(errno_num, err_msg, err_filename)          │
│   1822 │   │   │   │   raise child_exception_type(err_msg)                                       │
│   1823                                                                                           │
│   1824                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ ❱ 1 transcriber("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")        │
│   2                     

## ゼロショット分類
まず、ラベル付けされていないテキストを分類する必要がある、より困難なタスクに取り組むことから始めます。
これは実際のプロジェクトでよくあるシナリオです。
なぜなら、テキストにアノテーションを付けるのは通常時間がかかり、専門知識が必要だからです。
このような場合、ゼロショット分類パイプラインは非常に強力です。分類に使用するラベルを指定できるため、事前学習済みモデルのラベルに依存する必要がありません。


In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

## テキストの生成
パイプラインを使ってテキストを生成する方法を見てみましょう。主なアイデアは、プロンプトを与えると、モデルが残りのテキストを生成してそれをオートコンプリートすることです。これは、多くの携帯電話に搭載されている予測入力機能に類似しています。テキスト生成にはランダム性が含まれるため、結果は毎度変わる可能性があります。


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

## パイプラインでHubから任意のモデルを使用する

これまでの例では、タスクに応じたデフォルトのモデルを使用しましたが、特定のタスク（例えばテキスト生成）のパイプラインで使用するために、ハブから特定のモデルを選択することも可能です。モデルハブにアクセスし、左側にある対応するタグをクリックすると、そのタスクでサポートされているモデルのみが表示されます。このようなページが表示されるはずです。

distilgpt2モデルを試してみましょう。先ほどと同じパイプラインでロードする方法を説明します。



In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

## 穴埋め
次に試すパイプラインは、fill-maskです。このタスクのアイデアは、与えられたテキストの空白を埋めることです。
top_k 引数は、表示させたい可能性の数を制御します。ここでは、モデルはしばしばマスクトークンと呼ばれる特別な $<mask>$ワードを埋めることに注意してください。
マスクトークンはモデルによってこことなります。日本語のBERTモデルであるcl-tohoku/bert-base-japanese-whole-word-maskingでは、$[MASK]$です。

In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

## 固有表現認識
名前付き実体の認識（NER）は、入力テキストのどの部分が人物、場所、組織などの実体に対応するかをモデルが見つけるタスクです。

ここでは、モデルはSylvainが人（PER）、Hugging Faceが組織（ORG）、Brooklynが場所（LOC）であることを正しく識別しています。

パイプラインの作成関数でgrouped_entities=Trueオプションを渡すと、同じエンティティに対応する文の部分をまとめ直すようにパイプラインに指示します。ここでは、名前が複数の単語で構成されていても、モデルは「Hugging」と「Face」を1つの組織として正しくグループ化します。

In [ ]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

## 質問応答

このパイプラインは、与えられた文脈からの情報を使って質問に答えます。
このパイプラインは与えられたコンテキストから情報を抽出することで動作することに注意してください。


In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

## 要約

文章中の重要な部分をすべて（あるいはほとんど）維持したまま、より短い文章にする処理です。抽出型と生成型がありますが、ここでは基本的に生成型を扱います。
テキスト生成と同様に、結果にmax_lengthまたはmin_lengthを指定することができます。


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

## 翻訳
翻訳の場合、タスク名に言語ペアを指定すればデフォルトのモデルを使うこともできますが（「translation_en_to_fr」のように）、最も簡単な方法はモデルハブで使いたいモデルを選ぶことです。ここでは、フランス語から英語への翻訳を試してみます。


In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.")

# pipelineの仕組み

まずは完全な例として、先ほどのコードを実行したときに裏で何が起きていたかを見てみましょう。

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

このパイプラインは、「前処理」「モデルへの入力」「後処理」の3つのステップをグループ化したものです。

![図](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)

## トークナイザー（Tokenizer）を使った前処理
他のニューラルネットワークのように、Transformerのモデルは生のテキストを直接処理できないので、我々のパイプラインの最初のステップは、テキスト入力をモデルが理解できる数値に変換することです。これを行うために我々はトークナイザーを使います。トークナイザーは次のことを行います。

- 入力を単語、サブワード、またはトークンと呼ばれる記号（句読点など）に分割します。
- 各トークンを整数に対応させます。
- モデルにとって有用と思われる追加の入力を追加します。

この前処理はモデルがプリトレーニングされたときと全く同じ方法で行われる必要があるため、まずHubから情報をダウンロードする必要があります。そのために、AutoTokenizer クラスとその from_pretrained() メソッドを使用します。モデルのチェックポイント名を使用すると、モデルのトークナイザーに関連するデータを自動的に取得し、キャッシュします（したがって、以下のコードを最初に実行したときのみダウンロードされます）。

sentiment-analysis pipeline のデフォルトのチェックポイントは distilbert-base-uncased-finetuned-sst-2-english なので、以下を実行します。
*チェックポイントは、学習済みモデルを指します。パラメタだけでなくハイパーパラメータなども保存しています。


In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

トークナイザーに文章を直接渡すと、モデルへの入力データが戻ってきます。

あとは入力IDのリストをテンソルに変換するだけです。

TransformerはバックエンドにどのMLフレームワークを使うかを気にせずに使うことができます。
しかし、Transformerのモデルはテンソルしか入力として受け付けません。
返したいテンソルの種類（PyTorch、TensorFlow、あるいは素のNumPy）を指定するために、return_tensors引数を用います。


In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

パディングや切り捨てについては後で説明します。
ここで覚えておくべきことは、1つの文か文のリストを渡すことと、返したいテンソルの型を指定することです（型を渡さない場合は、結果としてリストのリストが得られます）。

出力自体は input_ids と attention_mask の2つのキーを含むdictです。 input_ids には2行の整数値（各文章に1つずつ）が含まれ、これは各文章中のトークンの一意の識別子です。attention_maskが何であるかはこの章の後半で説明します。

## モデルを見る
トークナイザーで行ったのと同じ方法でプリトレインされたモデルをダウンロードすることができます。Transformers は from_pretrained() メソッドも持つ AutoModel クラスを提供します。

In [ ]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

このコードでは、以前パイプラインで使用したのと同じチェックポイントをダウンロードし（実際には既にキャッシュされているはずです）、それを使ってモデルをインスタンス化しています。

このアーキテクチャには基本的なTransformerモジュールだけが含まれており、いくつかの入力が与えられると、hidden state（featuresとも呼ばれます）を出力します。各モデルの入力に対して、Transformerモデルによるその入力の文脈的理解を表す高次元ベクトルを取得することになります。

hidden stateはそれ自体で役に立つこともありますが、通常はヘッドと呼ばれるモデルの別の部分への入力となります。異なるタスクは同じアーキテクチャで実行されたかもしれませんが、これらのタスクにはそれぞれ異なるヘッドが関連付けられます。

### 高次元のベクトル？

Transformerモジュールが出力するベクトルは、通常大きなものです。一般に3つの次元を持ちます。

- Batch size: 一度に処理する配列の数(この例では2)
- Sequence length: シーケンスの数値表現の長さ(この例では16)。
- Hidden size: 各モデル入力のベクトル次元。

最後の値から「高次元」と言われる。Hidden sizeは非常に大きくすることができます（小さいモデルでは768が一般的で、大きいモデルでは3072以上に達することもあります）。

前処理をした入力をモデルに与えてみると、このことがわかります。

In [ ]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

Transformersモデルの出力は名前付きタプルか辞書のように動作することに注意してください。属性（上の例）かキー（outputs["last_hidden_state"]）、あるいは探しているものがどこにあるか正確に知っている場合はインデックスによって要素にアクセスできます（outputs[0])。

### Model heads: 数値の意味を理解する


モデルヘッドはHidden stateの高次元ベクトルを入力とし、それらを異なる次元（クラス分類ならクラス数）に変換します。モデルヘッドは通常1層または数層の線形層で構成されます。

![fig](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg)

Transformerモデルの出力は直接モデルヘッドに送られ、処理されます。

この図では、モデルはそのEmbedding layerとそれに続く層で表現されています。Embedding layerは、トークン化された入力の各入力IDを、関連するトークンを表すベクトルに変換します。
後続の層はアテンション機構を用いてそれらのベクトルを操作し、最終的な文の表現を生成します。

トランスフォーマーには多くの異なるアーキテクチャがあり、それぞれが特定のタスクに取り組むために設計されています。
これらは一部です。

- Model (Hidden stateを取り出す)
- ForCausalLM
- ForMaskedLM
- ForMultipleChoice
- ForQuestionAnswering
- ForSequenceClassification
- ForCausalLM
などです。

今回の例では、シーケンス分類のヘッドを持つモデルが必要になります（文章をポジティブかネガティブかに分類できるようにするため）。
そのため、実際にはAutoModelクラスではなく、AutoModelForSequenceClassificationを使用します。

In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

ここで、出力の形状を見ると、次元がかなり低くなっています。モデルヘッドは、先ほどの高次元ベクトルを入力とし、2つの値を含むベクトル（ラベルあたり1つ）を出力します。

In [ ]:
print(outputs.logits.shape)

2つの文と2つのラベルがあるだけなので、このモデルから得られる結果は2×2の形をしている。

## 出力の後処理
モデルから出力される値は、それ自体は理解不能です。

In [ ]:
print(outputs.logits)

このモデルは、最初の文は[-1.5607, 1.6123]、2番目の文は[ 4.1692, -3.3464]と予測しました。
これは確率ではなく対数であり、モデルの最終層が出力した正規化されていない生のスコアです。
確率に変換するためには、SoftMax layerを通過する必要があります。

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

ここで、モデルは最初の文章を[0.0402, 0.9598]、2番目の文章を[0.9995, 0.0005]と予測したことがわかります。これらが認識可能な確率のスコアです。

各ポジションに対応するラベルを得るには、モデル設定のid2label属性を調べます（これについては次のセクションで詳しく説明します）。

In [ ]:
model.config.id2label

これで、このモデルは次のように予測したというができます。

1. ネガティブ: 0.0402, ポジティブ: 0.9598
2. ネガティブ: 0.9995, ポジティブ: 0.0005

これで、パイプラインの3つのステップ、トークナイザーによる前処理、モデルへの入力の通過、そして後処理がうまく再現できました。

では、それぞれのステップをより深く掘り下げてみましょう。

# Models

ここではチェックポイントから任意のモデルをインスタンス化する際に便利な AutoModel クラスを使用します。

AutoModel クラスとその関連クラスは、ライブラリで利用可能な様々なモデルに対する単純なラッパーです。チェックポイントに適したモデルアーキテクチャを自動的に推測し、そのアーキテクチャを持つモデルをインスタンス化することができるラッパーです。

しかし、使いたいモデルの種類がわかっているのであれば、そのアーキテクチャを定義したクラスを直接使うことができます。BERTモデルでこれがどのように機能するかを見てみましょう。

### トランスフォーマーの作成

BERTモデルを初期化するために最初にしなければならないことは、configオブジェクトをロードすることです。


In [ ]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)

コンフィギュレーションには、モデルを構築するために使用される多くの属性が含まれています。

In [ ]:
print(config)

hidden_size 属性は hidden_states ベクターのサイズを定義し、num_hidden_layers は Transformer モデルが持つレイヤーの数を定義しています。

### 異なる読み込み方法

デフォルトの設定からモデルを作成すると、ランダムな値で初期化されます。

In [ ]:
from transformers import BertConfig, BertModel

config = BertConfig()
model = BertModel(config)

# Model is randomly initialized!

この状態でもモデルは使えますが、学習していないので、ちんぷんかんぷんな出力になってしまいます。手元のタスクに対して一からモデルを学習させることもできますが、これには長い学習時間と多くのデータが必要で、環境負荷も無視できないものになります。不必要で重複した作業を避けるためには、すでに学習されたモデルを共有し、再利用できることが不可欠です。

すでに学習済みのTransformerモデルを読み込むのは簡単です。from_pretrained()メソッドを使ってこれを行うことができます。

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

先ほど見たように、BertModel を同等の AutoModel クラスに置き換えることができます。これはチェックポイントに依存しないコードを生成するためで、あるチェックポイントでコードが動作すれば、他のチェックポイントでも動作するはずです。

これはアーキテクチャが異なっていても、チェックポイントが同様のタスク (例えばセンチメント分析タスク) に対して学習されたものである限り、動きます。

上記のコードサンプルではBertConfigを使用せず、代わりにbert-base-cased識別子で事前に学習したモデルをロードしています。これは、BERTの作者自身によって訓練されたモデルのチェックポイントです。それについての詳細は、そのモデルカードに記載されています。HuggingFace Hubの機能です。

このモデルは現在、チェックポイントのすべての重みで初期化されています。これは、訓練されたタスクの推論に直接使用することができ、また、新しいタスクでファインチューニングすることができます。ゼロからではなく、事前に学習させた重みを用いて学習させることで、迅速に良い結果を得ることができます。

重みはダウンロードされ、キャッシュフォルダ（デフォルトは ~/.cache/huggingface/transformers ）に保存されます。環境変数 HF_HOME を設定することで、キャッシュフォルダをカスタマイズすることができます。

モデルをロードするために使用される識別子は、BERTアーキテクチャと互換性がある限り、モデルハブ上の任意のモデルの識別子を使用することができます。利用可能なBERTチェックポイントの全リストは、ここで見つけることができます。

### 保存方法
モデルの保存はロードするのと同じくらい簡単です - from_pretrained()メソッドに類似したsave_pretrained()メソッドを使用します。


In [ ]:
model.save_pretrained("model")

これにより、2つのファイルがディスクに保存されます。

In [ ]:
ls model

config.json ファイルを見てみると、モデルアーキテクチャを構築するために必要な属性がわかると思います。このファイルには、チェックポイントがどこで発生したか、最後にチェックポイントを保存したときに使用していたTransformersのバージョンなど、いくつかのメタデータも含まれています。

pytorch_model.bin ファイルはstate dictionaryとして知られており、モデルのすべての重みが含まれています。この2つのファイルは密接に関係しています。コンフィギュレーションはモデルのアーキテクチャを知るために必要で、一方モデルの重みはモデルのパラメータです。

### 推論のためにTransformerモデルを使う

モデルをロードして保存する方法がわかったので、それを使って予測をしてみましょう。

トランスフォーマーは数字（トークナイザーが生成する数字）しか処理できません。トークナイザーについて説明する前に、モデルがどのような入力を受け入れるかを探ってみましょう。

トークンナイザーは入力を適切なフレームワークのテンソルに変換してくれますが、何が起こっているかを理解するために、入力をモデルに送る前に行わなければならないことを簡単に見てみましょう。

例えば、いくつかのシーケンスがあるとします。単語に見えますが、シークエンスです。

In [ ]:
sequences = ["Hello!", "Cool.", "Nice!"]

トークナイザーはこれらを語彙のインデックスに変換し、一般にinput IDと呼ばれます。各シーケンスは数字の羅列になっています。結果として出力されるのは

In [ ]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

これは符号化された配列のリスト：リストのリストです。テンソルは長方形の形しか受け付けないです。この「配列」はすでに長方形なので、テンソルに変換するのは簡単です。

In [ ]:
import torch

model_inputs = torch.tensor(encoded_sequences)

### モデルへの入力としてテンソルを使う
モデルでテンソルを使うのは非常に簡単で、入力でモデルを呼び出すだけです。

In [ ]:
output = model(model_inputs)

モデルは多くの異なる引数を受け取りますが、必要なのはinput IDだけです。他の引数が何をするのか、いつ必要なのかは後で説明しますが、まずはTransformerのモデルが理解できる入力を構築するトークナイザーを詳しく見てみましょう。

# トークナイザー
トークナイザー（Tokenizer）は、自然言語処理パイプラインの中核となるコンポーネントの1つです。その目的はただ一つ、テキストをモデルで処理できるデータに変換することです。モデルは数値しか処理できないので、トークナイザーがテキスト入力を数値データに変換する必要があります。このセクションでは、トークン化パイプラインで何が行われているかを具体的に説明します。

自然言語処理タスクでは、一般に処理されるデータは生テキストです。以下はそのようなテキストの例です。

In [ ]:
"Jim Henson was a puppeteer"

しかし、モデルは数字しか処理できないので、生のテキストを数字に変換する方法を見つける必要があります。これを行うのがトークナイザーであり、その方法はたくさんあります。目標は、最も意味のある表現、つまりモデルにとって最も意味のある表現、そして可能であれば最も小さい表現を見つけることです。

それでは、アルゴリズムの例をいくつか見ていきましょう。

## 単語ベースのトークナイザー

最初に思い浮かぶトークナイザーのタイプは、単語ベース（Word base）です。一般に、いくつかのルールを設定するだけで非常に簡単に使用でき、多くの場合、適切な結果を得ることができます。たとえば、以下の画像では、生のテキストを単語に分割し、それぞれの数値表現を見つけることが目的です。

![fig](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/word_based_tokenization.svg)

テキストを分割する方法はさまざまです。例えば、Pythonのsplit()関数を適用して空白を使い、テキストを単語にトークン化することができます。

In [ ]:
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

また、単語トークナイザーには、句読点に関する特別なルールを持つものもあります。この種のトークナイザーを使うと、かなり大きな「語彙」を作ることができます。語彙は、コーパスに含まれる独立したトークンの総数で定義されます。

各単語には0から始まり、語彙数に応じたIDが付与されます。モデルはこのIDを使って各単語を識別する。

単語ベースのトークナイザーで一つの言語を完全にカバーしようとすると、その言語の各単語に識別子が必要になり、膨大な量のトークンが生成されることになります。例えば、英語には50万語以上の単語があるので、各単語から入力IDへのマップを構築するには、それだけのIDを記録しておく必要があります。また、「dog」のような単語と「dogs」のような単語は表現が異なるため、モデルは最初「dog」と「dogs」が似ていることを知ることができず、無関係な単語として認識してしまいます。同じことが、"run "と "running "のような他の類似した単語にもあてはまり、モデルは初期状態では類似しているとは考えずに関連のない全く別の単語として扱います。

最後に、語彙にない単語を表現するためのカスタムトークンが必要です。これは「unknown」トークンとして知られており、しばしば「[UNK]」や「"」と表現されます。トークナイザーがこのようなトークンを大量に生成している場合は、一般に悪い兆候です。これは、その単語の適切な表現を取り出すことができず、途中で情報が失われているためです。語彙を作成するときの目標は、トークンナイザーがunknownのトークンをできるだけ少ない単語でトークン化するような方法で行うことです。

unknownのトークンの量を減らす方法の1つは、文字ベースのトークナイザーを使用して、1段階深く掘り下げることです。

## 文字ベースのトークナイザー

文字ベースのトークナイザー（Character base）は、テキストを単語ではなく、文字に分割します。これには主に2つの利点があります。

- 語彙が非常に少なくなる。
- すべての単語が文字から構成されるため、語彙外の（未知の）トークンが非常に少なくなります。

しかし、ここでもスペースや句読点に関する問題が発生してしまいます。

![fig](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/character_based_tokenization.svg)

言葉ではなく文字で表現するため、直感的に意味が薄れるという意見があります。しかし、これも言語によって異なり、例えば中国語では、ラテン語の文字よりも一文字一文字が持つ情報量が多いのです。日本語も同様ですね。

もうひとつ考慮すべきは、モデルで処理するトークンの種類は減るものの、トークンの量が非常に多くなることです。単語ベースのトークナイザーでは1つのトークンで済みますが、文字に変換すると10以上のトークンになることがよくあります。

両者の長所を活かすには、2つのアプローチを組み合わせた第3の手法、サブワードトークン化を使用することができます。

## サブワートによるトークナイズ

サブワードによるトークナイズ（サブワードトークン化アルゴリズム，Subword tokenization）は、よく使われる単語はより小さなサブワードに分割すべきではなく、稀な単語は意味のあるサブワードに分解すべきであるという原則に基づきます。

たとえば、「annouraingly」は希少な単語とみなされ、「annouraingly」と「ly」に分解される可能性があります。これらは両方とも単体のサブワードとしてより頻繁に出現する可能性があり、同時に「annouraingly」の意味は「annouraingly」と「ly」の合成意味によって維持されます。

以下は、サブワードトークン化アルゴリズムが "Let's do tokenization!" というシーケンスをどのようにトークン化するかを示す例です。

![fig](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/bpe_subword.svg)

例えば、上記の例では、「tokenization」は「token」と「ization」という2つのトークンに分割され、スペース効率が良く（長い単語を表すのに2つのトークンしか必要ない）、意味を持つようになったのです。これにより、少ないボキャブラリーで比較的良好なカバレッジを確保し、未知のトークンをほぼゼロにすることができます。

その他にも
当然のことながら、世の中にはもっとたくさんのテクニックがあります。

- GPT-2で使用されているバイトレベルBPE
- BERTで使用されているWordPiece
- SentencePieceまたはUnigram（いくつかの多言語モデルで使われている、SentencePieceはMecabの作者と同じ。Gooogleのエンジニア。）

API を使い始めるために必要なトークンナイザーの仕組みについて基本的なことを紹介しました。

## トークナイザーの読込と保存
トークナイザーのロードと保存は、モデルの場合と同様に簡単です。実際、from_pretrained() と save_pretrained() という同じ二つのメソッドに基づきます。これらのメソッドは、トークナイザーによって使用されるアルゴリズム (モデルのアーキテクチャのようなもの) とそのボキャブラリー (モデルの重みのようなもの) をロードまたは保存します。

BERT と同じチェックポイントで学習した BERT トークナイザーをロードすることは、BertTokenizer クラスを使用することを除いて、モデルをロードするのと同じ方法で行われます。

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

AutoModel と同様に、AutoTokenizer クラスはチェックポイント名に基づいてライブラリ内の適切なトークナイザークラスを取得し、任意のチェックポイントで直接使用することが可能です。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

これで、前のセクションで示したように、トークナイザーを使うことができるようになりました。

In [ ]:
tokenizer("Using a Transformer network is simple")

トークナイザーの保存は、モデルの保存と同じです。

In [ ]:
tokenizer.save_pretrained("directory_on_my_computer")

token_type_idsとattention_maskのキーについては少し後で説明します。まず、input_idsがどのように生成されるかを見てみましょう。そのためには、トークナイザーの中間メソッドを見る必要があります。

## Encoding

テキストを数値に変換することをエンコードといいます。エンコーディングは、トークン化とinput IDへの変換という2段階のプロセスで行われます。

これまで見てきたように、最初のステップはテキストを単語（または単語の一部、句読点など）に分割することで、これは通常トークンと呼ばれる。この処理には複数のルールがあり、モデルの事前学習時に使用されたのと同じルールを使用するために、モデルの名前を使用してトークナイザーをインスタンス化する必要があるのはそのためです。

第二のステップは、トークンを数値に変換し、テンソルを構築し、モデルに与えることです。これを行うために、tokenizer は vocabulary を持っています。これは from_pretrained() メソッドでインスタンス化する際にダウンロードする部分です。ここでも、モデルが事前学習されたときに使用されたのと同じ語彙を使用する必要があります。

この2つのステップをよりよく理解するために、別々に説明します。なお、トークン化パイプラインの一部を個別に実行するメソッドをいくつか使って、それらのステップの中間結果を示しますが、実際には（セクション 2 で示したように）入力に対してトークナイザを直接呼び出す必要があります。

## Tokenize
トークン化の処理は、トークンナイザーの tokenize() メソッドによって行われます。



In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

このメソッドの出力は、文字列（トークン）のリストである。

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

このトークナイザーはサブワード型のトークナイザーで、語彙で表現できるトークンを得るまで単語を分割します。ここではtransformerがそうで、transformと##erの2つのトークンに分割されます。

## トークンからinput IDへ
入力IDへの変換はconvert_tokens_to_ids()トークナイザーメソッドで処理されます。

In [ ]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

これらの出力は、一度適切なフレームワークテンソルに変換されると、本章の前半で見たように、モデルの入力として使用することができます。

## Decoding
デコードはその逆で、語彙のインデックスから文字列を得ようとするものである。これは、以下のようにdecode()メソッドで行うことができます。

In [ ]:
ecoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

デコードメソッドはインデックスをトークンに戻すだけでなく、同じ単語の一部であったトークンをグループ化して、読みやすい文章を生成していることに注意しましょう。この動作は新しいテキストを予測するモデル（プロンプトから生成されたテキスト、または翻訳や要約のようなシーケンスからシーケンスへの変換）を使用する際に非常に有用となります。

# 複数シーケンスの処理

これまでは、最も単純なユースケースである、長さの小さな1つのシーケンスに対する推論を行いました。しかし、実応用しようとすると、すぐにいくつかの疑問が浮かび上がってきます。

- 複数の配列をどのように扱うか？
- 長さの異なる複数の配列をどのように扱うのか？
- 語彙インデックスだけがモデルをうまく機能させるための入力なのだろうか？
- 長すぎる配列というのはあるのだろうか？

これらの疑問がどのような種類の問題を引き起こすのか、そしてTransformers APIを使ってどのように解決できるのかを見てみましょう。

## モデルは一括入力を想定している
前の演習では、シーケンスがどのように数値のリストに変換されるかを見ました。この数値のリストをテンソルに変換して、モデルに入力してみましょう。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
# This line will fail.
model(input_ids)

なぜ失敗したのでしょうか？"セクション2 "のパイプラインからのステップに従いました。

問題は、トランスフォーマーのモデルはデフォルトで複数の文章を期待するのに対し、私たちは単一のシーケンスをモデルに送ったことです。ここで私たちは、トークンナイザーをシーケンスに適用するときに、トークンナイザーが舞台裏で行うすべてのことを行おうとしました。しかしよく見ると、トークナイザーはinput IDのリストをテンソルに変換しただけでなく、その上に次元を追加していることがわかります。


In [ ]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

もう一度、新たな次元に挑戦して、input IDと結果のlogitsを出力してみます。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

バッチ処理とは、複数の文章を一度にモデルを通して送信することです。文章が1つしかない場合は、1つのシーケンスでバッチを構築すればよいのです。
バッチ処理では、複数の文章を与えてもモデルが動作するようにします。複数のシーケンスを使用することは、1つのシーケンスでバッチを構築するのと同じように簡単です。しかし、2つ目の問題があります。2つ（またはそれ以上）の文章をバッチ処理しようとすると、文章の長さが異なる場合がある。テンソルを扱ったことがある人なら、テンソルは長方形でなければならないので、input IDのリストを直接テンソルに変換することはできないです。この問題を回避するために、Paddingがあります。

## 入力のパディング
以下のリストはテンソルに変換することができないです。

In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

これを回避するために、パディングを使ってテンソルの形状を長方形にします。パディングは、長さの足りない文にパディングトークンと呼ばれる特別な単語を追加することで、すべての文が同じ長さになるようにするものです。例えば、10個の単語からなる文が10個、20個の単語からなる文が1個あるとすると、パディングはすべての文が20個の単語を持つことを保証します。この例では、結果としてテンソルは次のようになります。

In [ ]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

パディングトークンIDは、tokenizer.pad_token_idで確認できます。それを使って、2つの文を個別に、そしてまとめてモデルを通して送信してみましょう。

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

バッチ予測のロジットがおかしい：2行目は2番目の文のロジットと同じはずなのに、全く違う値になっています。

Transformerモデルの重要な特徴が、各トークンを文脈化するアテンション層にあるためです。これらはシーケンスの全てのトークンにアテンションしてしまうので、パディングトークンも考慮することになります。異なる長さの個々の文をモデルに通すとき、もしくは同じ文とパディングを適用したバッチを通すときに同じ結果を得るには、パディングトークンを無視するようにアテンション層に指示する必要があります。これはアテンションマスクを使用することで実現されます。

## アテンションマスク
アテンションマスク（Attention masks）はinput IDテンソルと全く同じ形状のテンソルであり、0と1で埋められます。1は対応するトークンに注目すべきことを示し、0は対応するトークンに注目すべきでない（すなわち、モデルの注目層によって無視されるべき）ことを示します。

前の例をアテンションマスクで完成させましょう。

In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

今度は、バッチ内の2番目の文について同じロジットを得ることができました。

2つ目のシーケンスの最後の値がパディングIDであり、アテンションマスクでは0値であることに注目してください。

## シーケンス長の長いデータの処理
Transformerのモデルでは、モデルに渡すことのできるシーケンスの長さには限界があります。
ほとんどのモデルは512か1024トークンまでのシーケンスを扱いますが、それ以上のシーケンスを処理するように要求されるとクラッシュします。この問題に対する解決策は2つあります。

- より長い配列長をサポートするモデルを使用する。
- 配列を切り詰める。

モデルによってサポートする配列の長さが異なり、中には非常に長い配列を専門に扱うものもあります。Longformerはその一例で、他にはLEDがあります。

そうでない場合は、max_sequence_lengthパラメータを指定して、配列を切り詰めることが必要です。

In [ ]:
max_sequence_length = 64
sequence = sequence[:max_sequence_length]

# 前処理の一括処理
私たちはほとんどの作業を手作業で行おうと頑張っています。私たちはトークナイザーがどのように動作するかを調べ、トークン化、入力IDへの変換、パディング、切り捨て、そしてアテンションマスクについて見てきました。

Transformers API は、ここで掘り下げる高レベルの関数で、これらすべてを処理することができます。文に対して直接トークナイザーを呼び出すと、モデルを通過する準備ができた入力が返ってきます。

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

ここで、model_inputs 変数には、モデルがうまく動作するために必要なすべてのものが含まれています。DistilBERTの場合、アテンションマスクと同様に入力IDが含まれます。追加の入力を受け入れる他のモデルも、トークナイザーオブジェクトによってそれらが出力されます。

以下のいくつかの例で見るように、このメソッドは非常に強力です。まず、1つのシーケンスをトークン化することができます。

In [ ]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

また、APIを変更することなく、一度に複数のシーケンスを処理することができます。

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)

いくつかの目的に応じてpaddingすることができます。

In [ ]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")

# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

また、配列を切り捨てることも可能です。

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

トークナイザーオブジェクトは、特定のフレームワークのテンソルへの変換を処理し、それをモデルに直接送信することができます。例えば、以下のコードでは、tokenizerが異なるフレームワークのテンソルを返すように促しています。"pt "はPyTorchのテンソル、"tf "はTensorFlowのテンソル、"np "はNumPyの配線を返しています。


In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# Returns TensorFlow tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")

## 特殊トークン
トークナイザーが返す入力IDを見てみると、先ほどとは少し違っていることがわかります。

In [ ]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

最初に1つ、最後に1つ、トークンが追加されました。これがどういうことかというと、上の2つのIDの並びを解読してみましょう。

In [ ]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

トークナイザーは先頭に[CLS]、末尾に[SEP]という特殊な単語を追加しています。これは、モデルがこれらの単語で事前学習されているためで、推論で同じ結果を得るためには、これらの単語も追加する必要があります。モデルによっては、特殊な単語を追加しないものや、異なる単語を追加するものもあります。また、これらの特殊な単語を最初だけ、あるいは最後だけに追加するモデルもあります。いずれにせよ、トークナイザーはどの単語が期待されているかを知っており、あなたのためにこれを処理します。

## まとめ：トークナイザーからモデルへ

さて、トークン化オブジェクトがテキストに適用されるときに使う個々のステップをすべて見てきましたが、最後にもう一度、トークン化オブジェクトのメイン API で複数のシーケンス（パディング！）、非常に長いシーケンス（トランケーション！）、複数のタイプのテンソルを処理できるかを見てみましょう。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

# データの処理フロー
シーケンス分類器を1バッチで学習させる方法を紹介します。

In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

もちろん、2つの文章でモデルを学習させただけでは、あまり良い結果は得られないでしょう。より良い結果を得るためには、より大きなデータセットを用意する必要があります。

ここでは、William B. DolanとChris Brockettの論文で紹介されたMRPC（Microsoft Research Paraphrase Corpus）データセットを例として使用します。このデータセットは5,801組の文からなり、両文が言い換えかどうか（つまり、両文が同じ意味かどうか）を示すラベルが付けられています。

## ハブからデータセットを読み込む

ハブにはモデルだけでなく、様々な言語の複数のデータセットがあります。ここでデータセットを閲覧することができます。MRPC データセットはGLUEベンチマークを構成する10個のデータセットのうちの1つで、10種類のテキスト分類タスクにおけるMLモデルの性能を測定するために使われる学術的なベンチマークです。

🤗 Datasetsライブラリは、データセットをダウンロードしてHubにキャッシュするための非常にシンプルなコマンドを提供します。MRPCデータセットはこのようにダウンロードすることができます。

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

ご覧のように、トレーニングセット、検証セット、テストセットを含む DatasetDict オブジェクトが得られます。それぞれ、いくつかの列（文1、文2、ラベル、idx）と、各セットの要素数である可変長の行が含まれています（つまり、トレーニングセットには3668組の文があり、バリデーションセットには408組、テストセットには1725組の文があるということです）。

このコマンドはデータセットをダウンロードし、キャッシュします。デフォルトでは ~/.cache/huggingface/datasets にあります。

raw_datasetsオブジェクトの各文章のペアに、辞書のようにインデックスを付けてアクセスすることができます。

In [ ]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

ラベルはすでに整数であることがわかるので、前処理をする必要はありません。どの整数がどのラベルに対応するかを知るには、raw_train_dataset の特徴量を調べます。これにより、各カラムの型がわかります。

In [ ]:
raw_train_dataset.features

裏側では，label は ClassLabel 型で，ラベル名と整数の対応付けが names フォルダに格納されています．0がnot_equivalent，1がequivalentに対応しています。

## データセットの前処理

データセットを前処理するために、テキストをモデルが理解できる数値に変換する必要があります。先ほど確認したように、これはトークナイザーを使って行われます。トークナイザーには1つの文または文のリストを与えることができるので、次のように各ペアのすべての最初の文とすべての2番目の文を直接トークン化することができます。

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

しかし、単に2つの配列をモデルに渡して、2つの文が言い換えであるかどうかの予測を得ることはできないです。2つの配列をペアとして扱い、適切な前処理を施す必要があるのです。幸いなことに、トークナイザーはシーケンスのペアを受け取り、BERTモデルが期待する方法でそれを準備することができます。

In [ ]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

input_ids と attention_mask について先ほど説明しましたが、token_type_ids については後回しにしました。この例では、これは入力のどの部分が最初の文であり、どの部分が2番目の文であるかをモデルに伝えるものです。

input_idsの中のIDをデコードして単語に戻すと、

In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

2つの文がある場合、モデルは入力が [CLS] sentence1 [SEP] sentence2 [SEP] という形式になることを期待していることがわかります。これをtoken_type_idsに合わせると以下のようになります。

In [ ]:
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[      0,      0,    0,     0,       0,          0,   0,       0,      1,    1,     1,        1,     1,   1,       1]

ご覧のように、入力のうち [CLS] sentence1 [SEP] に対応する部分はすべてtoken type idが0であり、sentence2 [SEP] に対応する他の部分はすべてtoken type idが1であることがわかります。

別のチェックポイントを選択した場合、トークン化された入力にtoken_type_idがあるとは限らないことに注意してください(例えば、DistilBERTモデルを使用する場合、それらは返されません)。それらは、モデルが事前学習中にそれらを見たので、それらをどうすればよいかを知っている場合にのみ返されます。

ここで、BERTはtoken_type_idで事前学習され、最初に話したマスクされた言語モデリング目的の上に、次の文の予測と呼ばれる追加の目的があります。このタスクの目的は、文のペア間の関係をモデル化することです。

次の文の予測では、モデルは文のペア（ランダムにマスクされたトークン）を与えられ、2番目の文が最初の文に続くかどうかを予測するよう求められる。このタスクが自明でないように、半分は抽出された元の文書で文が互いに続き、残りの半分は2つの異なる文書から2つの文が来るようにします。

一般に、トークン化された入力に token_type_id があるかどうかを気にする必要はありません。トークンナイザーとモデルで同じチェックポイントを使用する限り、トークンナイザーはモデルに何を提供すべきかを知っているので、すべてうまくいきます。

前の章のように、最初の文のリストを与え、次に2番目の文のリストを与えることによって、トークンナイザーに文のペアのリストを与えることができます。これは、パディングとトランケーションのオプションとも互換性があります。つまり、学習データセットの前処理を行う方法の1つは次のように書けます。



In [ ]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

これはうまくいくのですが、dictを返すという欠点があります (キーが input_ids, attention_mask, token_type_ids で、値がリストのリストである)。また、トークン化の際にデータセット全体を保存するのに十分なRAMがある場合のみ動作します（一方、🤗データセットライブラリのデータセットはディスクに保存されたApache Arrowファイルなので、要求したサンプルだけをメモリ上にロードしておくことができます）。

データをデータセットとして保持するために、Dataset.map() メソッドを使用します。これは、トークン化だけでなく、より多くの前処理が必要な場合にも柔軟に対応することができます。map() メソッドは、データセットの各要素に関数を適用して動作します。そこで、入力をトークン化する関数を定義してみましょう。

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

この関数はdict（データセットの項目など）を受け取り、input_ids, attention_mask, token_type_ids をキーとする新しいdictを返します。トークナイザーが文のペアのリストで動作するため、サンプルのdictが複数のサンプル（文のリストとして各キー）を含む場合にも動作することに注意してください（前に見たとおりです）。これにより、map() の呼び出しで batched=True オプションを使用できるようになり、トークン化の速度が大幅に向上します。このトークナイザーは、TokenizersライブラリのRustで書かれたトークナイザーによって支えられています。このトークナイザーは非常に高速ですが、それは一度にたくさんの入力を与えた場合のみです。

今のところ、トークン化関数では padding 引数を省いていることに注意してください。これは、すべてのサンプルを最大長にパディングするのは効率が悪いからです。バッチを構築するときにサンプルをパディングする方がよいでしょう。そうすれば、データセット全体の最大長ではなく、そのバッチの最大長にパディングすればよいのですから。これは、入力の長さが非常に変化する場合に、多くの時間と処理能力を節約することができます。

以下は、トークン化関数をすべてのデータセットに一度に適用する方法です。map の呼び出しで batched=True を使用しているので、関数はデータセットの複数の要素に一度に適用され、各要素には適用されません。これにより、前処理をより高速に行うことができます。

この処理をDatasetsライブラリが適用する方法は、前処理関数が返す辞書のキーごとに新しいフィールドをデータセットに追加することです。

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

map() で前処理関数を適用する際に num_proc 引数を渡せば、マルチプロセシングを使用することもできます。ここでは、Tokenizers ライブラリがすでに複数のスレッドを使用してサンプルのトークン化を高速化しているため、これを行いませんでしたが、このライブラリに支えられた高速トークナイザーを使用していない場合は、前処理を高速化することができます。

tokenize_function は input_ids, attention_mask, token_type_ids をキーとするdictを返すので、この3つのフィールドがデータセットのすべての分割に追加されます。もし前処理関数が、map()を適用したデータセットの既存のキーに対して新しい値を返した場合、既存のフィールドも変更することができることに注意してください。

最後に、すべての例をまとめて一番長い要素の長さになるようにパディングする必要があります。

## 動的なパディング処理

バッチ内のサンプルをまとめる関数はcollate関数と呼ばれます。これはDataLoaderをビルドするときに渡す引数で、デフォルトではサンプルをPyTorchテンソルに変換して連結する（要素がリスト、タプル、辞書の場合は再帰的に）だけの関数になっています。今回のケースでは、入力がすべて同じサイズではないので、これは不可能です。そこで、パディングをあえて遅らせ、各バッチで必要な分だけパディングを行い、パディングの多い長い入力にならないようにしました。

実際にこれを行うには、バッチ処理したいデータセットのアイテムに正しい量のパディングを適用するcollate関数を定義する必要があります。幸いなことに、TransformersライブラリはDataCollatorWithPaddingによってそのような関数を提供してくれています。これはインスタンス化するときにトークナイザーを受け取り（どのパディングトークンを使うか、そしてモデルがパディングを入力の左側と右側のどちらに期待するかを知るため）、必要なことをすべてやってくれます。

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

これをテストするために、トレーニングセットからバッチ処理したいサンプルをいくつか取ってきてみましょう。ここでは、idx, sentence1, sentence2 のカラムは不要であり、文字列を含むので削除し（文字列でテンソルを作ることはできない）、バッチ内の各エントリの長さを見てみることにします。

In [ ]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

当然ながら、32 から 67 までの様々な長さのサンプルが得られます。ダイナミックパディングとは、このバッチ内のサンプルはすべて、バッチ内の最大長である67になるようにパディングされるべきであるという意味です。ダイナミックパディングがなければ、すべてのサンプルはデータセット全体の最大長、つまりモデルが受け入れることのできる最大長にパディングされなければならないでしょう。data_collator がバッチを動的に正しくパディングしていることを再確認してみましょう。

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

いい感じ これで生テキストからモデルが扱えるバッチになったので、次はFine tuningです！

# Trainer APIを使ったファインチューニング

🤗 TransformersはTrainerクラスを提供し、あなたのデータセットで提供する事前学習済みモデルの微調整を手助けしてくれます。前節でデータの前処理をすべて終えたら、あとは Trainer を定義するためのいくつかのステップを残すのみです。一番大変なのは、Trainer.train()を実行する環境を準備することでしょう。CPUでは非常に遅く実行されるためです。

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Training
Trainerを定義する前の最初のステップはTrainingArgumentsクラスを定義することです。このクラスにはTrainerが学習と評価に使用する全てのハイパーパラメータが含まれています。唯一の引数は学習されたモデルを保存するディレクトリと、途中のチェックポイントです。それ以外はデフォルトのままでも構いませんし、基本的なファインチューニングであれば問題ないでしょう。

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

2番目のステップは、モデルを定義することです。前の章と同様、AutoModelForSequenceClassification クラスを使用し、2つのラベルを使用します。

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

この事前学習済みモデルをインスタンス化した後に警告が出ることがあります。これは、BERTが文のペアを分類するための事前学習を行っていないためで、事前学習されたモデルのヘッドは破棄され、代わりにシーケンス分類に適した新しいヘッドが追加されています。警告は、いくつかの重みが使用されなかったこと（削除された事前学習ヘッドに対応するもの）と、他のいくつかの重みがランダムに初期化されたこと（新しいヘッドのためのもの）を示しています。最後に、モデルを訓練するよう促していますが、これはまさに今私たちがしようとしていることです。

モデルを作成したら、モデル、training_args、学習・検証用データセット、data_collator、tokenizerなど、これまでに作成したすべてのオブジェクトを渡してTrainerを定義することができます。

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

なお、今回のようにトークナイザーを渡すと、Trainerが使うデフォルトのdata_collatorは前回定義したDataCollatorWithPaddingになるので、この呼び出しではdata_collator=data_collatorの行は飛ばしても大丈夫です。
データセットでモデルを微調整するために、Trainerのtrain()メソッドを呼び出すだけです。

In [ ]:
trainer.train()

これにより、微調整が開始され（GPUで数分かかる）、500ステップごとにトレーニングロスが報告されます。しかし、モデルがどの程度うまくいっているのか（あるいはうまくいっていないのか）についてはわかりません。これは、以下の理由からです。

Evalation_strategyを "steps"（eval_steps毎に評価）または "epoch"（各エポック終了時に評価）に設定し、トレーニング中に評価するようにTrainerに指示しなかった。
評価中に指標を計算するための compute_metrics() 関数を Trainer に提供していない（そうしないと、評価は単に損失を表示するだけで、あまり直感的な数値ではありません）。

## Evaluation：モデルの精度を評価する
どのようにして便利な compute_metrics() 関数を作成し、学習時に使用するかを見てみましょう。この関数はEvalPredictionオブジェクト（predictionフィールドとlabel_idsフィールドを持つ名前付きタプル）を受け取り、文字列とfloatをマッピングした辞書を返します（文字列は返されるメトリックスの名前で、floatはその値です）。このモデルからいくつかの予測を得るために、Trainer.predict()コマンドを使用することができます。


In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

predict()メソッドの出力は、predictions, label_ids, metricsの3つのフィールドを持つ別の名前付きタプルになります。metricsフィールドには、渡されたデータセットに対する損失と、いくつかの時間メトリクス（予測にかかった時間、合計と平均）が格納されるだけです。compute_metrics() 関数を完成させて Trainer に渡すと、そのフィールドには compute_metrics() が返すメトリクスも格納される。

見ての通り、predictionsは408 x 2の形状の2次元配列です（408は使用したデータセットの要素数です）。これらは predict() に渡したデータセットの各要素の対数です（前の章で見たように、すべての Transformer のモデルは対数を返します）。これらをラベルと比較できる予測値に変換するために、第2軸の最大値を持つインデックスを取る必要があります。

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

これで、これらの予測をラベルと比較することができます。compute_metric()関数を構築するために、Evaluateライブラリのメトリクスに依存することになります。MRPCデータセットに関連するメトリクスは、データセットをロードしたのと同じように、今度はevaluate.load()関数で簡単にロードすることができます。返されたオブジェクトは、メトリックの計算を行うために使用できるcompute()メソッドを持っています。

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

モデルヘッドのランダムな初期化により、達成したメトリクスが変わる可能性があるため、得られる正確な結果は異なるかもしれません。ここでは、我々のモデルが検証セットで85.78%の精度を持ち、F1スコアが89.97であることがわかります。この2つの指標は、GLUEベンチマークのMRPCデータセットで結果を評価するために使用されます。BERTの論文の表では、ベースモデルのF1スコアは88.9と報告されています。これはケースなしのモデルで、現在はケースありのモデルを使用しているため、より良い結果になっています。

すべてをまとめると、compute_metrics()関数ができあがります。

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

そして、各エポック終了時にメトリクスを報告するために実際に使われている様子を見るために、このcompute_metrics()関数で新しいTrainerを定義する方法を紹介します。

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

TrainingArgumentsのevaluation_strategyを "epoch "に設定し、新しいモデルを作成していることに注意してください。新しいトレーニングの実行を開始するために、以下を実行します。

In [ ]:
trainer.train()

今回は、トレーニングロスに加え、各エポック終了時に検証ロスとメトリクスを報告します。

Trainerは複数のGPUやTPUですぐに動作し、混合精度トレーニング（トレーニング引数でfp16 = Trueを使用）など、多くのオプションを提供します。